In [27]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [54]:
from proj1_helpers import *
from validation_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [52]:
from implementations import *

## Do your thing crazy machine learning thing here :) ...

In [30]:
split_x, split_y, split_ids = separate(y, tX, ids)

In [31]:
selection = dataStatistics(split_x)

Statistics 
Type :
0
[0.26145747 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.
 1.         1.         1.         1.         1.         0.        ]
Statistics 
Type :
1
[0.09751883 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         1.         0.        ]
Statistics 
Type :
2
[0.05859584 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
Statistics 
Type :
3
[0.0

In [32]:
cleaned = removeNone(split_x, selection)



# Now we can either drop the lines with residual Nones or replace the Nones by the median value

In [33]:
cleaned_with_median = putMedianInsteadOfNone(cleaned)

cleaned_with_median_with_momentum = add_momentum_vector(cleaned_with_median)

In [34]:
dropped_x, dropped_y, dropped_ids = dropLineIfNone(cleaned, split_y, split_ids)


## At this point, the first values in each of the split data has a PRI_jet_num = 0, then 1 and so on. The data is clean and we can work with it.

## Generate predictions and save ouput in csv format for submission:

In [35]:
DATA_TEST_PATH = '../data/test.csv' 
y_donotUse, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [55]:
#separate data with respect to column 24 and remove None
split_x_test, _, split_ids_test =  separate(y_donotUse, tX_test, ids_test)


split_x_cleaned_test = removeNone(split_x_test, dataStatistics(split_x_test))

#median instead of None
split_x_with_median = putMedianInsteadOfNone(split_x_cleaned_test)

split_x_with_median_with_momentum = add_momentum_vector(split_x_with_median)

#line dropped when None
#split_x_drop_lines, split_y_dropped_split_indexes_dropped = dropLineIfNone(split_x_cleaned_test, _, split_ids_test)

#degrees for polynomial feature expension
degrees = [1,2,3,4,5,6,7,8,9,10,11,12,13]

y_res = []

acc = []




for i in range(len(cleaned_with_median)):
    
    
    
    #training: chose either cross calidation or cross validation for logistic regression with regularization
    #w_star, d, accuracy, training_set = crossValidation_with_loss(cleaned_with_median[i], split_y[i], 0.98, degrees ,6)
    w_star, d, accuracy, training_set = crossValidationForLogistic_reg_with_loss(cleaned_with_median[i], split_y[i], 0.9, degrees ,6)
    
    
    #polynomial feature expension and normalization using the training data
    mean = np.mean(build_poly(training_set,d), axis = 0)
    std = np.std(build_poly(training_set,d), axis = 0)
    
      
    #put 1 if std = 0
    std = std + (std == 0)
    
    extended_and_normalized = (build_poly(split_x_with_median[i], d) - mean) / std
    
    #adding bias term
    bias = np.ones(shape=split_x_with_median[i].shape)          
    x_test_ready = np.c_[bias, extended_and_normalized]
    
    #prediction
    y_res.append(predict_labels(w_star, x_test_ready))


    acc.append(accuracy)

print("Accuracy per jet nbr: \n")
print(acc)


Statistics 
Type :
0
[0.2605448 0.        0.        0.        1.        1.        1.
 0.        0.        0.        0.        0.        1.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        1.        1.        1.        1.        1.
 1.        0.       ]
Statistics 
Type :
1
[0.09834149 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         1.         0.        ]
Statistics 
Type :
2
[0.05881481 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
Statistics 
Type :
3
[0.06376737 0.         0.     

lambda=0.10010,degree=6, gamma=0.61000, Training Loss=inf, Testing Loss=-245077698004456.44
lambda=0.10010,degree=6, gamma=0.91000, Training Loss=inf, Testing Loss=-248987955536784.56
lambda=0.10010,degree=7, gamma=0.01000, Training Loss=inf, Testing Loss=-15013792152402.926
lambda=0.10010,degree=7, gamma=0.31000, Training Loss=inf, Testing Loss=-215597888810504.88
lambda=0.10010,degree=7, gamma=0.61000, Training Loss=inf, Testing Loss=-245607151431493.7
lambda=0.10010,degree=7, gamma=0.91000, Training Loss=inf, Testing Loss=-249526638238589.8
lambda=0.10010,degree=8, gamma=0.01000, Training Loss=inf, Testing Loss=-15066792291478.574
lambda=0.10010,degree=8, gamma=0.31000, Training Loss=inf, Testing Loss=-216372955481398.3
lambda=0.10010,degree=8, gamma=0.61000, Training Loss=inf, Testing Loss=-246496744426732.6
lambda=0.10010,degree=8, gamma=0.91000, Training Loss=inf, Testing Loss=-250432294241944.25
lambda=0.10010,degree=9, gamma=0.01000, Training Loss=inf, Testing Loss=-15092155136

lambda=0.00010,degree=3, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=3, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=3, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=3, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=4, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=4, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=4, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=4, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=5, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=5, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=5, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=5, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=6, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.000

lambda=0.20010,degree=4, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=4, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=4, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=4, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=7, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.200

lambda=0.10010,degree=5, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=5, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=5, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=5, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=6, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=6, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=6, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=6, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=7, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=7, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=7, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=7, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.10010,degree=8, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.100

lambda=0.00010,degree=5, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=5, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=5, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=6, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=6, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=6, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=6, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=7, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=7, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=7, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=7, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=8, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.00010,degree=8, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.000

lambda=0.20010,degree=4, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=5, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=6, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=7, gamma=0.01000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=7, gamma=0.31000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=7, gamma=0.61000, Training Loss=inf, Testing Loss=inf
lambda=0.20010,degree=7, gamma=0.91000, Training Loss=inf, Testing Loss=inf
lambda=0.200

In [56]:
OUTPUT_PATH = '../data/submission.csv'
#reassemble the data for the submission
y_pred = put_together(y_res, split_ids_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)